# Independent Scheduler Example

This notebook demonstrates how to create an even contribution schedule for sinking fund envelopes using the Independent Scheduler. The Independent Scheduler creates smooth, predictable contribution patterns for each bill without considering interactions between different bills.

The workflow shown here represents a typical sinking fund management process:
1. Load bills from a data source
2. Allocate any existing balance to the appropriate envelopes
3. Create a contribution schedule to fully fund each bill by its due date

The Independent Scheduler is ideal when you want clear visibility into how much you're contributing to each bill over time, with consistent contribution amounts for each individual bill.

## Imports

In [1]:
########################################################################
## FOR NOTEBOOKS ONLY: ADD THE PROJECT ROOT TO THE PYTHON PATH
########################################################################

import os
import sys

sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), '..'))
)

# Imports.
import datetime
from sinkingfund.allocation.sorted import SortedAllocator
from sinkingfund.schedules.indep_scheduler import IndependentScheduler
from sinkingfund.models.bills import Bill
from sinkingfund.models.envelope import Envelope

## Creating a Set of Bills and Envelopes

In [2]:
bills = [
    {
        'bill_id': 'car_insurance',
        'service': 'Insurance',
        'amount_due': 800,
        'recurring': True,
        'start_date': datetime.date(2026, 4, 24),
        'frequency': 'monthly',
        'interval': 6
    },
    {
        'bill_id': 'car_registration',
        'service': 'Registration',
        'amount_due': 300,
        'recurring': True,
        'start_date': datetime.date(2026, 5, 1),
        'frequency': 'annual',
        'interval': 1
    },
    {
        'bill_id': 'property_taxes',
        'service': 'Taxes',
        'amount_due': 5000,
        'recurring': True,
        'start_date': datetime.date(2026, 11, 1),
        'frequency': 'annual',
        'interval': 1
    }
]

bills = [Bill(**b) for b in bills]
envelopes = [Envelope(bill=b, interval=14) for b in bills]

## Allocate the Balance

Allocate any existing balance using the sorted allocation and the cascade sort key.

In [3]:
# The start date of the contribution schedule. The current balance must
# match the balance at the start date. If there are unpaid bills in the
# account, you manually need to remove their amounts from the balance.
start_date = datetime.date(2025, 10, 25)
balance = 1000

In [4]:
# Initialize the allocator.
allocator = SortedAllocator(sort_key='cascade')

# Allocate the balance to the envelopes.
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

# Verify total allocated equals budget.
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)
print([(e.bill.bill_id, e.bill.amount_due, e.allocated, e.remaining) for e in envelopes])

True
[('car_insurance', 800, 800, 0), ('car_registration', 300, 200, 100), ('property_taxes', 5000, 0, 5000)]


## Creating Contribution Schedules

Now we use the Independent Scheduler to create even contribution schedules for each envelope.

In [5]:
# Initialize the scheduler.
scheduler = IndependentScheduler()

# Schedule the contributions.
scheduler.schedule(envelopes=envelopes, start_date=start_date)

# Verify (1) that the total contributions equal to the amount remaining;
# (2) that the total contributions plus the initial allocation equals
# the amount due.
for e in envelopes:
    
    if e.schedule is None:
        continue

    contrib = sum(cf.amount for cf in e.schedule if cf.amount > 0)
    print(e.bill.bill_id, contrib == e.remaining, contrib, e.remaining)
    print(e.bill.bill_id, contrib + e.allocated == e.bill.amount_due, contrib, e.remaining)

car_insurance True 0 0
car_insurance True 0 0
car_registration True 100.0 100
car_registration True 100.0 100
property_taxes True 5000.0 5000
property_taxes True 5000.0 5000


## Examining the Results

The scheduler creates a series of cash flows for each envelope.

Notice how the Independent Scheduler:
- Creates evenly-sized contributions (except potentially the first one).
- Spaces them at the specified interval (14 days).
- Adds a final negative cash flow representing the bill payment.
- Ensures the total contributions exactly equal the remaining amount needed.

This predictable, regular contribution pattern makes budget planning straightforward and gives you clear visibility into how you're progressing toward each financial goal.

In [8]:
envelopes[1].schedule

[CashFlow(bill_id='car_registration', date=datetime.date(2025, 10, 25), amount=3.54),
 CashFlow(bill_id='car_registration', date=datetime.date(2025, 11, 8), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2025, 11, 22), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2025, 12, 6), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2025, 12, 20), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 1, 3), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 1, 17), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 1, 31), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 2, 14), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 2, 28), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 3, 14), amount=7.42),
 CashFlow(bill_id='car_registration', date=datetime.date(2026, 